In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdDistGeom

def mol_to_xyz(smiles_list):
    results = [] 
    
    # Iterate over each SMILES string in the list
    for smiles in smiles_list:
        # SMILES to RDKit
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, rdDistGeom.ETKDG()) # 3D coordinates 
        num_atoms = mol.GetNumAtoms() # RDKit XYZ string
        xyz_lines = [str(num_atoms)]
        xyz_lines.append(smiles)
        for atom in mol.GetAtoms():
            symbol = atom.GetSymbol()
            conf = mol.GetConformer()
            pos = conf.GetAtomPosition(atom.GetIdx())
            xyz_lines.append(f"{symbol} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}")
        
        # Append the XYZ to results list
        results.append("\n".join(xyz_lines))

    return results


In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdDistGeom
from morfeus import read_xyz, XTB

def get_molecules(smiles_list, N):
    xyz_contents = mol_to_xyz(smiles_list)# SMILES list to XYZ 
    n_nucs_max = []
    n_eles_max = []
    for i, xyz_content in enumerate(xyz_contents):
        fukui_electrophilicity = []
        fukui_nucleophilicity = []
        for _ in range(N):
            xyz_file = f"molecule_{i}.xyz"  # XYZ to a file
            with open(xyz_file, 'w') as file:
                file.write(xyz_content)
            
            elements, coordinates = read_xyz(xyz_file) # Morfeus to read XYZ file
            
            xtb = XTB(elements, coordinates) # Create  XTB object 
            
            fukui_electrophilicity.append(xtb.get_fukui("electrophilicity"))
            fukui_nucleophilicity.append(xtb.get_fukui("nucleophilicity"))

        # Fukui electrophilicity max value
        max_values_electrophilicity = [max(f.values()) for f in fukui_electrophilicity]
        n_eles_max.append(sum(max_values_electrophilicity) / len(max_values_electrophilicity))

        # Fukui nucleophilicity max value
        max_values_nucleophilicity = [max(f.values()) for f in fukui_nucleophilicity]
        n_nucs_max.append(sum(max_values_nucleophilicity) / len(max_values_nucleophilicity))

    n_eles_max = dict(zip(smiles_list, n_eles_max))
    n_nucs_max = dict(zip(smiles_list, n_nucs_max))

    if nucl_elec == "E":
        print(n_nucs_max) #nucleophilicity max values
    else:
        print(n_eles_max) #electrophilicity max values

In [ ]:
user_input = input("Enter a list of SMILES strings separated by commas: ")
smiles_list_input = user_input.split(',')
smiles_list_input = [smiles.strip() for smiles in smiles_list_input]

nucl_elec = input("Should we be ranking according to electrophilicity(E) or nucleophilicity(N)?")

get_molecules(smiles_list_input, 400)

In [ ]:
CNO, CNC, CN, Nc1ccccc1

In [ ]:
 CC=O, CC(=O)C, O=COC, CN(C)C(C)=O